In [1]:
#!pip install keras_tuner

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
charity_numbers = pd.read_csv("charity_numbers.csv")
charity_numbers.head()

,Unnamed: 0,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0,1,5000,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,1,108590,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
2,2,1,5000,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,3,1,6692,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4,4,1,142590,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0


In [3]:
# Split our preprocessed data into our features and target arrays
y = charity_numbers['IS_SUCCESSFUL']
X = charity_numbers.drop(['IS_SUCCESSFUL'], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [4]:
charity_numbers['IS_SUCCESSFUL'].value_counts()

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64

In [5]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh', 'sigmoid', 'softmax'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=5), activation=activation, input_dim=46))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [31]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [32]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=50,validation_data=(X_test_scaled,y_test))

Trial 174 Complete [00h 01m 28s]
val_accuracy: 0.7265306115150452

Best val_accuracy So Far: 0.7357434630393982
Total elapsed time: 00h 48m 16s
INFO:tensorflow:Oracle triggered exit


In [33]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 21, 'num_layers': 4, 'units_0': 21, 'units_1': 16, 'units_2': 11, 'units_3': 11, 'units_4': 11, 'tuner/epochs': 50, 'tuner/initial_epoch': 17, 'tuner/bracket': 3, 'tuner/round': 3, 'tuner/trial_id': '0130'}
{'activation': 'tanh', 'first_units': 21, 'num_layers': 1, 'units_0': 16, 'units_1': 11, 'units_2': 26, 'units_3': 11, 'units_4': 11, 'tuner/epochs': 50, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 16, 'num_layers': 1, 'units_0': 21, 'units_1': 16, 'units_2': 6, 'units_3': 21, 'units_4': 11, 'tuner/epochs': 50, 'tuner/initial_epoch': 17, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0164'}


In [34]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5478 - accuracy: 0.7357 - 412ms/epoch - 2ms/step
Loss: 0.5478070378303528, Accuracy: 0.7357434630393982
268/268 - 0s - loss: 0.5488 - accuracy: 0.7334 - 377ms/epoch - 1ms/step
Loss: 0.5488115549087524, Accuracy: 0.7334110736846924
268/268 - 0s - loss: 0.5494 - accuracy: 0.7334 - 374ms/epoch - 1ms/step
Loss: 0.5493907928466797, Accuracy: 0.7334110736846924
